In [1]:
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, request
from werkzeug.wrappers import Request, Response
from datetime import timedelta, datetime

In [2]:
engine = create_engine('sqlite:///Resources/hawaii.sqlite', echo=False)

# Declare a Base using 'automap_base()'
Base = automap_base()

# Reflect Database into ORM classes
Base.prepare(engine, reflect=True)

# Save a reference to the measurenment table as 'Measurement'
Measurement = Base.classes.measurement

# Save a reference to the station table as 'Station'
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Select the last date recorded
latest = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
latest = datetime.strptime(latest, '%Y-%m-%d')
latest = (latest - timedelta(days = 365))

app = Flask(__name__)
@app.route("/")

def welcome():
   
    return (
        "<h1> Hawaii Weather Data</h1>"
        
        "<h3>Available routes below:</h3>"
        
        "Precipiation.<br/>"
        "/api/v1.0/precipitation<br/><br/>"
        
        "All Stations.<br/>"
        "/api/v1.0/stations<br/><br/>"
        
        "Temperature Observations for the last year.<br/>"
        "/api/v1.0/tobs<br/><br/>"
        
        "Find the temperature data between two dates between<br/>"
        "/api/v1.0/&ltstart&gt/&ltend&gt, for example, /api/v1.0/2017-01-01/2017-01-04"
        
    )


@app.route("/api/v1.0/precipitation")

def precipitation():

    # Retrieve the last 12 months of precipitation data
    results = session.query(Measurement.date, Measurement.prcp).\
                        filter(Measurement.date > begin_date).\
                        order_by(Measurement.date).all()

    # Create a dictionary from the row data and append to a list of for the precipitation data
    precipitation_data = []
    for prcp_data in results:
        prcp_data_dict = {}
        prcp_data_dict["Date"] = prcp_data.date
        prcp_data_dict["Precipitation"] = prcp_data.prcp
        precipitation_data.append(prcp_data_dict)
        

    return jsonify(precipitation_data)
@app.route("/api/v1.0/stations")

def stations():
    
    # Query all the stations
    results = session.query(Station).all()

    # Create a dictionary from the row data and append to a list of all_stations.
    all_stations = []
    for stations in results:
        stations_dict = {}
        stations_dict["Station"] = stations.station
        stations_dict["Station Name"] = stations.name
        stations_dict["Latitude"] = stations.latitude
        stations_dict["Longitude"] = stations.longitude
        stations_dict["Elevation"] = stations.elevation
        all_stations.append(stations_dict)
    
    return jsonify(all_stations)
@app.route("/api/v1.0/tobs")

def tobs():
    
    # Query all the stations and for the given date. 
    results = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
                    group_by(Measurement.date).\
                    filter(Measurement.date > latest).\
                    order_by(Measurement.station).all()
                    
    # Create a dictionary from the row data and append to a list of for the temperature data.
    temp_data = []
    for tobs_data in results:
        tobs_data_dict = {}
        tobs_data_dict["Station"] = tobs_data.station
        tobs_data_dict["Date"] = tobs_data.date
        tobs_data_dict["Temperature"] = tobs_data.tobs
        temp_data.append(tobs_data_dict)
    
    return jsonify(temp_data)

@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def start_end(start, end=latest):


    start_date_dt = dt.datetime.strptime(start, '%Y-%m-%d')
    if end == latest:
        end_date_dt = end
    else:
        end_date_dt = dt.datetime.strptime(end, '%Y-%m-%d')

    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    temps = session.query(*sel). \
                filter(Measurement.date>=start_date_dt). \
                filter(Measurement.date<=end_date_dt).all()[0]
    
    results_list = [{"temp_min": temps[0]}, 
                    {"temp_avg": temps[1]}, 
                    {"temp_max": temps[2]}]
    return jsonify(results_list)
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Aug/2019 18:23:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2019 18:23:32] "GET /favicon.ico HTTP/1.1" 404 -
[2019-08-19 18:23:39,200] ERROR in app: Exception on /api/v1.0/precipitation [GET]
Traceback (most recent call last):
  File "C:\Users\aotal\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aotal\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aotal\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aotal\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\aotal\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aotal\Anaconda3\lib\